In [1]:
import FinanceDataReader as fdr

sp500 = fdr.StockListing("S&P500")
sp500.head()

,Symbol,Name,Sector,Industry
0,MMM,3M,Industrials,Industrial Conglomerates
1,AOS,A. O. Smith,Industrials,Building Products
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment
3,ABBV,AbbVie,Health Care,Biotechnology
4,ACN,Accenture,Information Technology,IT Consulting & Other Services


In [2]:
# rsi값 만드는 함수

def make_rsi(close, periods = 14):
    delta = close.diff()
    gain = delta.where(delta > 0, 0).ewm(alpha = 1/periods, min_periods = periods, adjust = False).mean()     
    # 오른날만 데이터 가져오고 오르지 않는 날은 0으로 채움        # alpha값은 14일을 기준으로 구하므로 1/14.  최소기간은 14일을 지키도록
    loss = -delta.where(delta < 0, 0).ewm(alpha = 1/periods, min_periods = periods, adjust = False).mean()
    rs = gain / (loss + 1e-9)
    return 100 - (100 / (1 + rs))

df = fdr.DataReader("AAPL")
make_rsi(df["Close"])

1980-12-12          NaN
1980-12-15          NaN
1980-12-16          NaN
1980-12-17          NaN
1980-12-18          NaN
                ...    
2025-06-27    49.770113
2025-06-30    56.806887
2025-07-01    60.653082
2025-07-02    66.289029
2025-07-03    67.493758
Name: Close, Length: 11230, dtype: float64

In [3]:
from tqdm import tqdm
import pandas as pd

sp500_dic = {}
for i in tqdm(range(len(sp500))):
    try:
        row = sp500.iloc[i]
        symbol = row["Symbol"]
        name = row["Name"]
        df = fdr.DataReader(symbol)
        df['RSI'] = make_rsi(df['Close'])
        df['Change'] = df['Close'].pct_change() * 100           # 변화율
        df = df.dropna()
        sp500_dic[symbol] = [name, df]
    except:
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 502/502 [05:37<00:00,  1.49it/s]


#### 1일 후의 가격 예측

In [26]:
# 왜 특성을 추가해도 성능이 비슷한가?  왜 항상 좋게 나오는가?
# 데이터 누수문제 = 미래데이터가 데이터 학습 시에 영향을 미친다 

year = 2017
period = 1      # 1일 후의 가격을 예측하겠다

train_x = [] 
train_y = []
test_x = [] 
test_y = []

for symbol in sp500_dic:
    name = sp500_dic[symbol][0]
    df = sp500_dic[symbol][1].copy()[["Close", "Change", "RSI"]]    # 종가데이터, 변화율, RSI

    # 특정시점 기준으로 분리해서 데이터 섞이지 않게.... 데이터 누수문제 해결
    train = df[df.index.year < year].values
    test = df[df.index.year >= year].values

    for i in range(len(train) - period):
        a = train[i, 1:]   # Close값 빼고 Change,RSI만 가져옴
        before = train[i, 0]      # i날짜에 대한 종가
        after = train[i+period, 0]    # 그다음날(i+1)날짜에 대한 종가
        b = (after-before)/before*100 >= 5       # 1일 후 가격이 5% 이상 올랐는지 확인
        train_x.append(a)
        train_y.append(b)

    for i in range(len(test) - period):
        a = test[i, 1:]
        before = test[i, 0]
        after = test[i+period, 0]
        b = (after-before)/before*100 >= 5
        test_x.append(a)
        test_y.append(b)

In [27]:
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(train_x, train_y)

pred = model.predict(test_x)

report = classification_report(test_y, pred)
print(report)

# 정밀도 떨어짐 - 데이터 누수문제 해결된걸로 보임

              precision    recall  f1-score   support

       False       0.98      1.00      0.99   1030263
        True       0.08      0.00      0.00     16170

    accuracy                           0.98   1046433
   macro avg       0.53      0.50      0.50   1046433
weighted avg       0.97      0.98      0.98   1046433



#### 40일 후의 가격 예측

In [4]:
# 왜 특성을 추가해도 성능이 비슷한가?  왜 항상 좋게 나오는가?
# 데이터 누수문제 = 미래데이터가 데이터 학습 시에 영향을 미친다 

year = 2017
period = 40      # 40일 후의 가격을 예측하겠다

train_x = [] 
train_y = []
test_x = [] 
test_y = []

for symbol in sp500_dic:
    name = sp500_dic[symbol][0]
    df = sp500_dic[symbol][1].copy()[["Close", "Change", "RSI"]]    # 종가데이터, 변화율, RSI

    # 특정시점 기준으로 분리해서 데이터 섞이지 않게.... 데이터 누수문제 해결
    train = df[df.index.year < year].values
    test = df[df.index.year >= year].values

    for i in range(len(train) - period):
        a = train[i, 1:]   # Close값 빼고 Change,RSI만 가져옴
        before = train[i, 0]      # i날짜에 대한 종가
        after = train[i+period, 0]    # 그다음날(i+40)날짜에 대한 종가
        b = (after-before)/before*100 >= 5       # 40일 후 가격이 5% 이상 올랐는지 확인
        train_x.append(a)
        train_y.append(b)

    for i in range(len(test) - period):
        a = test[i, 1:]
        before = test[i, 0]
        after = test[i+period, 0]
        b = (after-before)/before*100 >= 5
        test_x.append(a)
        test_y.append(b)

In [5]:
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(train_x, train_y)

pred = model.predict(test_x)

report = classification_report(test_y, pred)
print(report)

# 40일 후를 예측하면 1일 후를 예측했을때에 비해 rsi가 변동될 시간이 더 있어서 정밀도 좀더올라감

              precision    recall  f1-score   support

       False       0.62      1.00      0.76    631294
        True       0.51      0.01      0.01    394005

    accuracy                           0.62   1025299
   macro avg       0.56      0.50      0.39   1025299
weighted avg       0.57      0.62      0.47   1025299

